In [1]:
# ### File to automate the data parsing of the output files (excel files of energies) and create the required graphs 
# and input-output csv file for ML network input

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from numpy import trapz
import numpy as np

M = 100 #Number of models

### Read job output files and creat the figures for energies and RF-U

for ii in range(1,M+1):
    data = pd.read_csv("Dynamic_Job-%d_Energies_RF_U.csv" %(ii))
    t = data["Time_steps"][:]
    AE = data["Artificial_Energy"][:]
    IE = data["Internal_Energy"][:]
    KE = data["Kinetic_Energy"][:]
    TE = data["Total_En"][:]
    U3 = data["Disp_Im"][:]
    RF = data["Contact_Normal_Force"][:]
    plt.plot(t, AE, color='b', label='Artificial Strain Energy')
    plt.plot(t, IE, color='g', label='Internal Energy')
    plt.plot(t, KE, color='r', label='Kinetic Energy')
    plt.plot(t, TE, color='k', label='Total Energy')
    plt.legend()
    plt.title("Energies - time")
    plt.xlabel('Time (s)')
    plt.ylabel('Energy (KJ)')
    # plt.savefig('Energies.png',transparent= True)
    plt.savefig('Job-%d_Energies.png' %(ii))
#     plt.show()
    plt.close()    # close the figure window
    plt.plot(U3,RF, color='violet', label='Reaction force of CB')
    plt.legend()
    plt.title("Reaction force - displacement")
    plt.xlabel('Displacement of impactor (mm)')
    plt.ylabel('Reaction force (N)')
    # plt.savefig('Energies.png',transparent= True)
    plt.savefig('Job-%d_RF_Disp.png' %(ii))
#     plt.show()  
    plt.close()    # close the figure window

In [3]:
### Calculate the peak impact force, energy absorbed, total deformation and mean impact force in every job file

# Two new dictionaries to store the peak impact force and energy abosorbed for every job
max_value = []
area = []
Deforamtion = []
Mean_force = []
CB_weight = []

for jj in range(1,M+1):
    data = pd.read_csv("Dynamic_Job-%d_Energies_RF_U.csv" %(jj))
    t = data["Time_steps"][:]
    IE = data["Internal_Energy"][:]
    RF = data["Contact_Normal_Force"][:]
    AE = data["Artificial_Energy"][:]
    U3 = data["Disp_Im"][:]
    mass_CB = data["CB_mass"][:]
    
    MV = round(RF[:50].max()/1000,5)
    max_value.append(MV)
#     print("Initial peak contact force for model-%d in KN = "%(jj) + str(MV))
    
    # PLOT = plt.plot(t, IE, label='IE-%d' %(jj))
    # plt.legend()
    # EA = round(trapz(IE,t),2)
    EA = round((IE[200]-AE[200])/1000000,5)
    area.append(EA)
#     print("Energy absorbed by Model-%d in KJ =" %(jj) + str(EA))
    
    d = round(U3.max(),4)-10
    Deforamtion.append(d)
#     print("Deformation of CB for model-%d in mm = "%(jj) + str(d))
    
    P_mean = round((EA*1000)/d,5)
    Mean_force.append(P_mean)
#     print("Mean impact force of CB for model-%d in KN = "%(jj) + str(P_mean))
    
    mass = mass_CB.max()*1000000
    CB_weight.append(mass)
#     print("Mass of CB for model-%d in grams = "%(jj) + str(mass))
#     print("")
 

In [4]:
# max_value

In [5]:
# area

In [6]:
# mv = np.array(max_value)
# a = np.array(area)

In [7]:
# DATA = np.hstack((mv,a))

In [8]:
# DATA

In [9]:
# read the previously generated file for model and it's parameters
df = pd.read_csv('model_database_dynamic_impact_sims.csv')

# Add columns in right to add the peak impact force and energy absorbed
df["Peak_Impact_Force_KN"] = max_value
df['Energy_Absorbed_KJ'] = area
df['Deformation_CB_mm'] = Deforamtion
df['Mean_impact_force_KN']= Mean_force
df['Mass_CB_g'] = CB_weight

In [10]:
df

,Model,Width,Height,Thickness,Length,Velocity,Mass_Im,Dynamic_Job,Time_req_s,Peak_Impact_Force_KN,Energy_Absorbed_KJ,Deformation_CB_mm,Mean_impact_force_KN,Mass_CB_g
0,Dy_Model-1,80.0,80.0,1.1,250.0,9300.0,0.1,Dynamic_Job-1,2958.979,23.50310,3.89610,238.5323,16.33364,238.0
1,Dy_Model-2,80.0,80.0,1.8,250.0,6300.0,0.1,Dynamic_Job-2,457.913,42.97311,1.88273,110.1169,17.09756,389.0
2,Dy_Model-3,80.0,80.0,1.5,250.0,7200.0,0.1,Dynamic_Job-3,534.983,34.75908,2.40167,189.9236,12.64545,324.0
3,Dy_Model-4,80.0,80.0,1.7,250.0,9000.0,0.1,Dynamic_Job-4,560.366,41.21080,3.76597,204.6034,18.40619,367.0
4,Dy_Model-5,80.0,80.0,1.1,250.0,5100.0,0.1,Dynamic_Job-5,429.849,23.38740,1.09032,155.8358,6.99660,238.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Dy_Model-96,80.0,80.0,1.4,250.0,6700.0,0.1,Dynamic_Job-96,525.010,30.88385,2.01997,185.1961,10.90720,302.0
96,Dy_Model-97,80.0,80.0,1.2,250.0,8400.0,0.1,Dynamic_Job-97,875.168,26.76667,3.17765,226.0244,14.05888,259.0
97,Dy_Model-98,80.0,80.0,1.4,250.0,5900.0,0.1,Dynamic_Job-98,459.122,29.35737,1.60477,153.4128,10.46047,302.0
98,Dy_Model-99,80.0,80.0,1.1,250.0,8300.0,0.1,Dynamic_Job-99,889.600,23.85514,3.10190,233.1330,13.30528,238.0


In [11]:
print(isinstance(df, dict))
print(df.keys())

False
Index(['Model', 'Width', 'Height', 'Thickness', 'Length', 'Velocity',
       'Mass_Im', 'Dynamic_Job', 'Time_req_s', 'Peak_Impact_Force_KN',
       'Energy_Absorbed_KJ', 'Deformation_CB_mm', 'Mean_impact_force_KN',
       'Mass_CB_g'],
      dtype='object')


In [12]:
# Save the new file in excel file with new name 'Model_Database'
df.to_excel('MODELS_DATABASE.xlsx', index=False)
df.to_csv('MODELS_DATABASE.csv', index=False)